# Photutils Overview

## Exercise solutions

## Exercise 1

`data/xdf_hst_wfc3ir_60mas_f105w_sci.fits` is an F105W image of the same field used for the preceding examples. These images are aligned, so source positions detected in the F160W image should correspond to the same source if they are visible in the F105W image. 

In the F105W image, find the countrate in a 5-pixel aperture radius of the 3 brightest sources detected in the F160W image (hint: read in the table of detected sources that was written to disk in the previous example)

In [ ]:
from astropy.io import fits
from astropy.table import Table
from photutils import CircularAperture, aperture_photometry

In [ ]:
# read in the data 
f105w_sci_hdulist = fits.open('data/xdf_hst_wfc3ir_60mas_f105w_sci.fits')
f105w_sci_data = f105w_sci_hdulist[0].data

# read in the table of sources detected in the F160W image
detected_sources = Table.read('my_detected_sources.txt', format='ascii.ecsv')

In [ ]:
# sort table by source_sum column and select the 3 brightest
detected_sources.sort('source_sum')
detected_sources.reverse()
three_brightest = detected_sources[0:3]

In [ ]:
# pull out x, y coordinates from table
x = three_brightest['xcentroid']
y = three_brightest['ycentroid']

# make list of tuples of coordinate pairs
positions = list(zip(x, y))
positions

In [ ]:
# construct 5-pixel radius circular aperture object
aperture = CircularAperture(positions, r=5)

# run aperture photometry
phot = aperture_photometry(f105w_sci_data, aperture)


In [ ]:
# inspect photometry table. All sources are brighter in F105W than they are in F160W
phot